# Implementing Latent Dirichlet Allocation for Topic Modeling and Document Classification

### Andrew Cooper, Brian Cozzi

## Abstract

In this report we implement a form of Latent Dirichlet Allocation for topic modeling. Latent Dirichlet Allocation (LDA) was first introduced by Blei, Ng, and Jordan in 2003 as a hierarchical modeling approach for discrete data such as text in a corpus. This algorithm hinges on the notion that collections of data, such as text in a document, are generated according to a latent topic distribution, where each topic assigns probabilities to different words. The purpose of LDA in topic modeling is to group documents based on similar topic distributions, and to identify key words in each topic. Using a collapsed Gibbs sampler approach to LDA as described in Darling 2011, we implement an algorithm that estimates the latent topic distributions of a given corpus of documents. In addition, our algorithm returns key words assigned to each topic. We optimize our algorithm's Gibbs sampler using "just-in-time" compilation. We then evaluate the performance of our algorithm on both simulated data and documents from the Newsgroups corpus. Finally, we compare the accuracy of our algorithm to a variational bayes approach to LDA and to Latent Semantic Analysis (LSA).

Key phrases: Latent Dirichlet Allocation, Topic Modeling, Collapsed Gibbs Sampler, Newsgroups Corpus, Variational Bayes, Latent Semantic Analysis

## Background (Brian)

State the research paper you are using. Describe the concept of the algorithm and why it is interesting and/or useful. If appropriate, describe the mathematical basis of the algorithm. Some potential topics for the background include:
* What problem does it address?
* What are known and possible applications of the algorithm?
* What are its advantages and disadvantages relative to other algorithms?
* How will you use it in your research?

## Description of Algorithm

The LDA algorithm takes in 4 inputs: a corpus of documents, the number of topics, two optional choices for hyperparameters, and an optional specification on the number iterations for the Gibbs Sampler. The ultimate goal of the algorithm is to estimate the topic distribution for each document as well as the word distribution for each topic. It does this by making inference on the latent topics of each word in the given corpus. We perform this inference by implementing a Gibbs sampler. First the algorithm sets a starting point by randomly sampling topics for each of the words in the corpus. Then it iteratively samples new topics for each word using calculated posterior probabilities. After a number of iterations, the Gibbs Sampler returns the estimated topics for each word, which are then used to calculate the latent topic distributions and word distributions using Monte Carlo estimation. These estimated quantities are returned to the user.

### Variables

The LDA algorithm has many different symbols and components. We establish all the symbols used in our algorithm below:

* $K$ = The number of topics
* $M$ = The number of documents in the corpus
* $N_m$ = The number of words in document $m$
* $V$ = The number of possible words
* $w_{m,n}$ = The nth word in document $m$
* $z_{m,n}$ = The nth topic in document $m$ 
* $\theta_m$ = The topic distribution of document $m$
* $\phi_k$ = The word distribution of topic $k$

### Algorithm Input

The algorithm takes as input a corpus of documents represented as a $M$x$V$  bag-of-words matrix. In addition, it takes as input the number of topics $K$. It also takes in two positive values representing the hyperparameters for the topic distribution ($\alpha$) and the word distribution ($\beta$). For this implementation we use symmetric priors for the dirichlet distributions, which means only one value is needed as input for each prior. Finally it takes as input the number iterations for the Gibbs sampler.

### Gibbs Sampler

The algorithm for LDA has three main steps. The first step of the algorithm is preparing the data for the Gibbs sampler. As a starting point for our sampler, we first must randomly assign topics $z_{m,n}$ for each of the words in the given corpus. We then create two different count matrices, $N_1$ and $N_2$: $N_1$ is a $M$x$K$ matrix that counts the distribution of topics across documents. $N_2$ is a $K$x$V$ matrix that counts the distribution of words across topics. The count matrices are initialized according to the random topic assignment.

The second step is the implementation of the Gibbs sampler. For each iteration, our sampler loops through every word $w_{m,n}$ in every document of our corpus. For each word, we first remove its assigned topic and decrement the appropriate count matrices $N_1$, $N_2$, and $N_3$. We then calculate the posterior probabilities of the word belonging to each of the possible topics. One of these topics is sampled using these probabilities and is assigned to the word. Finally, all of the count matrices are incremented according to this new topic. This process is done for all the words in the corpus for how many iterations the user specified.

***Gibbs Sampler Implementation***

* Randomly assign topics $z_{m,n}$ for all words in corpus, and initialize count matrices $N_1$ and $N_2$

* **for** $i = 1$ to n_iter:
   * **for** $m = 1$ to $M$:
       * **for** $n = 1$ to $N_m$:
           * $w = w_{m,n}$
           * $z_{old} = z_{m,n}$
           * $N_1[m, z_{old}] -= 1$
           * $N_2[z_{old}, w] -= 1$
           * **for** $k = 1$ to $K$:
               * $p_k = Pr(z = k | \dots) \propto (N_1[m, k] + \alpha[k])*\dfrac{N_2[k, w] + \beta[w]}{\sum_V{N_2[k, v] + \beta[v]}}$
           * Normalize $p$ 
           * $z_{new} = $ Sample from $Cat(K, p)$
           * $z_{m,n} = z_{new}$
           * $N_1[m, z_{new}] += 1$
           * $N_2[z_{new}, w] += 1$

### Parameter Estimation

The third step to the algorithm is estimating and returning the quantities of interest. One quantity to estimate is the topic distribution $\theta_m$ for each document $m$. Another quantity to estimate is the word distribution $\phi_k$ for each topic. These quantities are estimated using the count matrices $N_1$ and $N_2$ established in the Gibbs sampler:

$\hat{\theta}_{m,k} = \dfrac{N_1[m, k] + \alpha[k]}{\sum_k{N_1[m, k] + \alpha[k]}}$ 

$\hat{\phi}_{k, v} = \dfrac{N_2[k, v] + \beta[v]}{\sum_v{N_2[k, v] + \beta[v]}}$

## Description of Performance Optimization

## Applications to Simulated Datasets

In the LDA framework, documents are assumed to be generated under the following stochastic process:

For each document $m$, sample topic distribution $ \theta_m \sim Dirichlet(\alpha)$

For each topic $k$, sample word distribution $ \phi_k \sim Dirichlet(\beta)$

For each word $n$ in each document,

1) Sample topic $z_n \sim Cat(\theta_m)$

2) Sample word $w_n \sim Cat(\phi_{z_n})$

To assess the correctness of our LDA algorithm, we simulate data under this stochastic process. We then train this data on our algorithm and compare the parameter estimates to the true parameters.

We simulate a corpus of 10 documents containing 100 unique "words". Documents in the corpus are composed of 2 different topics and contain between 150 and 200 words.

In [ ]:
# Load libraries
import numpy as np
from LDA_AandB.test_data_generator import simulate_corpus
from LDA_AandB.lda_code import lda, group_docs
np.random.seed(101)

In [ ]:
# Set corpus parameters
V = 100
N_min = 150
N_max = 200
K = 2
M = 10

# Set hyperparameters
alpha_true = np.random.randint(1, 10, K)
beta_true = np.random.randint(1, 10, V)

# Generate simulated corpus
bow, theta_true, phi_true = simulate_corpus(alpha_true, beta_true, M, N_min, N_max)

The accuracy of our LDA depends on the choice of the hyperparameters $\alpha$ and $\beta$. The closer these hyperparameters are to the true values of the dataset, the better the algorithm's estimates of the topic and word distributions. 

When the hyperparameters $\alpha$ and $\beta$ are chosen to be the true values, our LDA algorithm captures the true topic distributions very well.

In [ ]:
# Train data on LDA implementation
theta, phi = lda(bow, K, alpha_true, beta_true, 10000)
print("Average absolute error in topic distribution estimates:", np.mean(abs(theta - theta_true)))
print("LDA document groups:", group_docs(theta))
print("True document groups:", group_docs(theta_true))

However, in real-world scenarios we don't know what the true values of $\alpha$ and $\beta$ are. In the case where the chosen hyperparameters are $\textbf{not}$ the true values from the data, our LDA algorithm's estimates are less accurate.

In [ ]:
# Train data on LDA implementation
theta, phi = lda(bow, K, 1, 1, 10000)
print("Average absolute error in topic distribution estimates:", np.mean(abs(theta - theta_true)))
print("LDA document groups:", group_docs(theta))
print("True document groups:", group_docs(theta_true))

While our LDA algorithm performs correctly, the simulated data testing illustrates how choosing proper prior parameters for the model can severly affect acccuracy in parameter estimation.

## Applications to Real Dataset

We apply our algorithm to the Newsgroups corpus. This popular corpus contains documents from 30 different topics ranging from science to politics to religion.

For our analysis we choose 15 randomly chosen documents from the Newsgroups corpus under the categories "Computer Graphics" and "Christianity". We then assess how accurately our LDA algorithm classifies these documents.

In [ ]:
# Load libraries
import numpy as np
from LDA_AandB.lda_code import lda, group_docs, get_key_words
from LDA_AandB.test_data_generator import get_newsgroups, newsgroups_categories
np.random.seed(101)

In [ ]:
cats = [newsgroups_categories[i] for i in [1, 15]]
bow_news, labels, words = get_newsgroups(cats, 15)
print("Document Categories:", labels)

In [ ]:
theta, phi = lda(bow_news, 2, 1, 1, 10000)
group_docs(theta, 2)
labels
get_key_words(phi, 1, words)

## Comparative Analysis with Competing Algorithms

We compare our LDA algorithm to an alternative approach to LDA and to another method of document classification called latent semantic analysis, or LSA.

### Latent Dirichlet Allocation (Variational Bayes Approach)

The sklearn package in python implements LDA using the variational bayes approach as described in the original 2003 paper. The variational bayes approach introduces additional variational parameters to optimize. The algorithm minimizes the KL divergence between the posterior probability of the actual parameters and the posterior probability of the new variational parameters. The parameters are estimated using an Expectation-Maximization approach.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

lda = LatentDirichletAllocation(n_components = 2,
                                random_state = 0)
lda.fit(bow) 
LatentDirichletAllocation(...)

lda.transform(bow)

### Latent Semantic Analysis

LSA is a different approach to classification than LDA. In essence, LSA is an application of a singular value decomposition, or SVD.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components = 2, n_iter = 7, random_state = 42)
TruncatedSVD(algorithm = 'randomized', n_components = 2, n_iter = 7,
        random_state = 42, tol = 0.0)

svd.fit_transform(bow)

## Discussion/Conclusion

## References/Bibliography

Darling, W.M. (2011). A Theoretical and Practical Implementation Tutorial on Topic Modeling and Gibbs Sampling.

David M. Blei, Andrew Y. Ng, and Michael I. Jordan. 2003. Latent dirichlet allocation. J. Mach. Learn. Res. 3 (March 2003), 993-1022.